# Guide to Fine-Tuning TinyLlama on `jhana-question-answer` Dataset

# Setup

In [ ]:
!git clone https://github.com/OpenAccess-AI-Collective/axolotl/tree/main

%cd axolotl

# follow instructions in the README to install the dependencies: https://github.com/OpenAccess-AI-Collective/axolotl/tree/main

# create a new conda environment
!conda create -n jhana-axolotl python=3.10

pip3 install packaging

pip3 install -e '.[flash-attn,deepspeed]'

# i had some trouble with dependencies; these packages worked for me

"""
accelerate==0.26.1
addict==2.4.0
aiobotocore==2.12.0
art==6.1
bitsandbytes==0.42.0
bitsandbytes_cuda112==0.26.0.post2
colorama==0.4.6
deepspeed==0.13.1
einops==0.7.0
evaluate==0.4.1
fire==0.5.0
flash_attn==2.5.0
fschat==0.2.36
gcsfs==2024.2.0
gradio==4.19.2
huggingface_hub==0.21.3
Jinja2==3.1.3
lion_pytorch==0.1.2
mamba_ssm==1.2.0.post1
mlflow==2.11.0
modal==0.61.3
numba==0.59.0
numpy==1.26.4
optimum==1.16.2
pandas==2.2.1
peft==0.9.1.dev0
pydantic==2.6.3
pynvml==11.5.0
pytest==8.1.0
PyYAML==6.0.1
PyYAML==6.0.1
Requests==2.31.0
s3fs==2024.2.0
safetensors==0.4.2
setuptools==69.1.1
termcolor==2.4.0
tokenizers==0.15.2
torch==2.1.1
tqdm==4.66.2
transformers==4.39.0.dev0
wandb==0.16.3
xformers==0.0.23
"""

# modify the tiny-llama/lora.yml file

"""
base_model: TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T
model_type: LlamaForCausalLM
tokenizer_type: LlamaTokenizer

load_in_8bit: true
load_in_4bit: false
strict: false

datasets:
  #- path: mhenrichsen/alpaca_2k_test
  - path: carecodeconnect/jhana-question-answer
    type: alpaca
dataset_prepared_path:
val_set_size: 0.05
output_dir: ./jhana-tinyllama

sequence_len: 4096
sample_packing: false # was true
pad_to_sequence_len: true

adapter: lora
lora_model_dir:
lora_r: 32
lora_alpha: 16
lora_dropout: 0.05
lora_target_linear: true
lora_fan_in_fan_out:

wandb_project:
wandb_entity:
wandb_watch:
wandb_name:
wandb_log_model:

gradient_accumulation_steps: 4 # could increase to 8
micro_batch_size: 2 # could decrease to 1
num_epochs: 4
optimizer: adamw_bnb_8bit
lr_scheduler: cosine
learning_rate: 0.0002

train_on_inputs: false
group_by_length: false
bf16: auto
fp16:
tf32: false

gradient_checkpointing: true
early_stopping_patience:
resume_from_checkpoint:
local_rank:
logging_steps: 1
xformers_attention:
flash_attention: false

warmup_steps: 10
evals_per_epoch: 4
saves_per_epoch: 1
debug:
deepspeed:
weight_decay: 0.0
fsdp:
fsdp_config:
special_tokens:
"""

# Preprocess Data

In [ ]:
# preprocess data
!CUDA_VISIBLE_DEVICES=0 python -m axolotl.cli.preprocess examples/tiny-llama/lora.yml


# Train Model

In [ ]:
# train tiny-llama on the Jhana dataset
!accelerate launch -m axolotl.cli.train examples/tiny-llama/lora.yml

# Generate Text

In [ ]:
# run inference using axolotl terminal tool
!python -m axolotl.cli.inference examples/tiny-llama/lora.yml --lora_model_dir="./jhana-tinyllama"